In [1]:
!pip install optuna
!pip install imblearn
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import optuna



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df=pd.read_csv('clinvar.csv',low_memory=False)

In [3]:
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,NaN,not_specified,...,tolerated,benign,NaN,NaN,NaN,NaN,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",NaN,Spinocerebellar_ataxia_21|not_provided,...,deleterious_low_confidence,benign,NaN,NaN,NaN,NaN,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",NaN,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,...,deleterious,probably_damaging,NaN,NaN,NaN,NaN,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002...",NaN,Shprintzen-Goldberg_syndrome|not_provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002",NaN,Shprintzen-Goldberg_syndrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.700,4.766224,-3.0


In [4]:
df['PolyPhen'].value_counts()



PolyPhen
benign               13329
probably_damaging     7531
possibly_damaging     3932
unknown                  4
Name: count, dtype: int64

In [5]:
df = df[df['PolyPhen'].notna()].copy()
df['PolyPhen'] = df['PolyPhen']

In [6]:
# removing unkown and combining damaging ones(probably and possibly)
df = df[df['PolyPhen'] != 'unknown']
df['PolyPhen'].value_counts()

PolyPhen
benign               13329
probably_damaging     7531
possibly_damaging     3932
Name: count, dtype: int64

In [7]:
df['PolyPhen'] = df['PolyPhen'].apply(lambda x: 'benign' if x == 'benign' else 'possibly_damaging')
df['PolyPhen'].value_counts()

PolyPhen
benign               13329
possibly_damaging    11463
Name: count, dtype: int64

In [8]:
x=df.drop('PolyPhen',axis=1)
y=df['PolyPhen']

In [9]:
df = df[df['PolyPhen'] != 'unknown']
threshold = 0.42 * len(df)
df.dropna(axis=1, thresh=threshold, inplace=True)
display(df.head())

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Amino_acids,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,not_specified,NC_000001.10:g.1168180G>C,...,E/D,gaG/gaC,1.0,NaN,tolerated,benign,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",Spinocerebellar_ataxia_21|not_provided,NC_000001.10:g.1470752G>A,...,P/L,cCg/cTg,-1.0,OK,deleterious_low_confidence,benign,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",Strabismus|Nystagmus|Hypothyroidism|Intellectu...,NC_000001.10:g.1737942A>G,...,I/T,aTc/aCc,-1.0,OK,deleterious,possibly_damaging,NaN,28.100,6.061752,-1.0
9,1,5925304,G,A,0.0045,0.00231,0.0058,"MedGen:C0687120,Orphanet:ORPHA655,SNOMED_CT:20...",Nephronophthisis|not_specified,NC_000001.10:g.5925304G>A,...,T/M,aCg/aTg,-1.0,NaN,deleterious,benign,0.021,22.100,2.969650,-1.0
10,1,5926503,G,A,0.0019,0.00267,0.0010,"MedGen:C0687120,Orphanet:ORPHA655,SNOMED_CT:20...",Nephronophthisis|not_specified,NC_000001.10:g.5926503G>A,...,R/W,Cgg/Tgg,-1.0,NaN,deleterious,possibly_damaging,0.021,26.100,5.430178,-3.0


In [10]:
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

categorical_cols = [col for col in categorical_cols if col != 'PolyPhen']
print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)

Numerical columns: ['POS', 'AF_ESP', 'AF_EXAC', 'AF_TGP', 'ORIGIN', 'CLASS', 'STRAND', 'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'BLOSUM62']
Categorical columns: ['CHROM', 'REF', 'ALT', 'CLNDISDB', 'CLNDN', 'CLNHGVS', 'CLNVC', 'MC', 'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'BAM_EDIT', 'SIFT']


In [11]:
x = df[numerical_cols + categorical_cols]
x = df[numerical_cols + categorical_cols].copy()
for col in categorical_cols:
    x[col]=x[col].astype(str)

In [12]:
df.isnull().sum()

CHROM                  0
POS                    0
REF                    0
ALT                    0
AF_ESP                 0
AF_EXAC                0
AF_TGP                 0
CLNDISDB               0
CLNDN                  0
CLNHGVS                0
CLNVC                  0
MC                     0
ORIGIN                 0
CLASS                  0
Allele                 0
Consequence            0
IMPACT                 0
SYMBOL                 0
Feature_type           0
Feature                0
BIOTYPE                0
EXON                   0
cDNA_position          0
CDS_position           0
Protein_position       0
Amino_acids            0
Codons                 0
STRAND                 0
BAM_EDIT            9830
SIFT                 118
PolyPhen               0
LoFtool             1662
CADD_PHRED            43
CADD_RAW              43
BLOSUM62            5121
dtype: int64

In [13]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [15]:
num_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('processor',PowerTransformer(method='yeo-johnson')),
    ('scaler',StandardScaler()) ])

cat_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehotencoding',OneHotEncoder(drop='first',handle_unknown='ignore')) ])

In [16]:
processor=ColumnTransformer([
    ('numerical',num_pipeline,numerical_cols),
    ('categorical',cat_pipeline,categorical_cols)
])

In [17]:

def objective(trial):
    model_type = trial.suggest_categorical("model_type", ["rf", "dt"])

    if model_type == "rf":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 30, 100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 40),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "class_weight": trial.suggest_categorical("class_weight", ["balanced", {"benign": 1, "possibly_damaging": 5}]),
            "random_state": 42,
            "n_jobs": -1
        }
        clf = RandomForestClassifier(**params)

    else:  
        params = {
            "max_depth": trial.suggest_int("max_depth", 10, 100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 40),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
           "class_weight": trial.suggest_categorical("class_weight", ["balanced", {"benign": 1, "possibly_damaging": 5}]),
            "random_state": 42
        }
        clf = DecisionTreeClassifier(**params)

    pipeline = Pipeline([
        ('preprocessor', processor),
        ('clf', clf)
    ])

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x_train, y_train,
                             cv=cv, scoring='f1_macro', n_jobs=-1)

    return np.mean(scores)


study = optuna.create_study(direction='maximize', study_name='RF_DT_Search')
study.optimize(objective, n_trials=30)


print(" Best Model Type:", study.best_params['model_type'])
print(" Best Hyperparameters:", study.best_params)
print(" Best CV f1_macro Score:", study.best_value)



[I 2025-06-27 14:14:58,125] A new study created in memory with name: RF_DT_Search
C:\Users\DEV KHAPRA\AppData\Local\Programs\Python\Python313\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {'benign': 1, 'possibly_damaging': 5} which is of type dict.
  warnings.warn(message)
[I 2025-06-27 14:15:02,913] Trial 0 finished with value: 0.7731487163597963 and parameters: {'model_type': 'dt', 'max_depth': 89, 'min_samples_split': 22, 'min_samples_leaf': 5, 'criterion': 'entropy', 'class_weight': {'benign': 1, 'possibly_damaging': 5}}. Best is trial 0 with value: 0.7731487163597963.
C:\Users\DEV KHAPRA\AppData\Local\Programs\Python\Python313\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {'benign': 1, 'possibly_damaging

 Best Model Type: rf
 Best Hyperparameters: {'model_type': 'rf', 'n_estimators': 247, 'max_depth': 74, 'min_samples_split': 27, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'class_weight': 'balanced'}
 Best CV f1_macro Score: 0.8369637230027801


In [18]:
final_pipeline=Pipeline([
    ('preprocessor',processor),
    ('random forest',RandomForestClassifier(n_estimators=267,max_depth=76,min_samples_split=13,min_samples_leaf=1,max_features='sqrt',
                                             class_weight='balanced',random_state=42,verbose=True,n_jobs=-1))
])
final_pipeline.fit(x_train,y_train)
y_pred=final_pipeline.predict(x_test)

report=classification_report(y_test,y_pred)
print("accuracy on test set:",report)



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 267 out of 267 | elapsed:    9.0s finished
C:\Users\DEV KHAPRA\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 3, 4, 5, 7, 8, 11, 13, 15, 16, 17, 18, 19, 20, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s


accuracy on test set:                    precision    recall  f1-score   support

           benign       0.85      0.86      0.85      2662
possibly_damaging       0.83      0.82      0.83      2297

         accuracy                           0.84      4959
        macro avg       0.84      0.84      0.84      4959
     weighted avg       0.84      0.84      0.84      4959



[Parallel(n_jobs=8)]: Done 267 out of 267 | elapsed:    0.1s finished


In [19]:
import joblib

In [21]:
joblib.dump(final_pipeline,'final_pipeline.pkl')

['final_pipeline.pkl']

In [22]:
joblib.dump(final_pipeline,'C:\\Users\\DEV KHAPRA\\OneDrive\\Desktop\\jupyter notebook\\gene_mutation\\final_pipeline.pkl')

['C:\\Users\\DEV KHAPRA\\OneDrive\\Desktop\\jupyter notebook\\gene_mutation\\final_pipeline.pkl']

In [23]:
import sklearn
print( sklearn.__version__)

1.6.1


In [24]:
!pip install cloudpickle
import cloudpickle
import os


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
save_dir = r"C:\Users\DEV KHAPRA\OneDrive\Desktop\jupyter notebook\gene_mutation\models"
os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

file_path = os.path.join(save_dir, "final_pipelineY.pkl")

with open(file_path, "wb") as f:
    cloudpickle.dump(final_pipeline, f)